In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [2]:
df_Norway= pd.read_csv("df_Norway.csv" ,encoding='unicode_escape')
df_Norway

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Norway,2020/1/1,5421241,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Norway,2020/1/2,5421241,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Norway,2020/1/3,5421241,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Norway,2020/1/4,5421241,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Norway,2020/1/5,5421241,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,Norway,2021/9/24,5421241,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,1.0,5.0,2.0,0.004569
633,Norway,2021/9/25,5421241,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004556
634,Norway,2021/9/26,5421241,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004547
635,Norway,2021/9/27,5421241,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004531


In [3]:
#shift 21days mortality
df_Norway_mortality=df_Norway["mortality"]
df_Norway_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
         ...   
632    0.004569
633    0.004556
634    0.004547
635    0.004531
636    0.000000
Name: mortality, Length: 615, dtype: float64

In [4]:
#删除后21天
df_Norway.drop(df_Norway.index[616:637], inplace=True)
df_Norway

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Norway,2020/1/1,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Norway,2020/1/2,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Norway,2020/1/3,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Norway,2020/1/4,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Norway,2020/1/5,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,Norway,2021/9/3,5421241,83%,30920.0,7273050.0,5.657,1330.689,30617.0,5.602,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.004980
612,Norway,2021/9/4,5421241,83%,15887.0,7288937.0,2.907,1333.595,30519.0,5.584,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.004946
613,Norway,2021/9/5,5421241,83%,15821.0,7304758.0,2.895,1336.490,30409.0,5.564,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.004914
614,Norway,2021/9/6,5421241,83%,40079.0,7344837.0,7.333,1343.823,30243.0,5.533,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.004879


In [5]:
df_Norway1=df_Norway.join(df_Norway_mortality,how="left",rsuffix="after21days")
df_Norway1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,Norway,2020/1/1,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Norway,2020/1/2,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,Norway,2020/1/3,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,Norway,2020/1/4,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,Norway,2020/1/5,5421241,83%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,Norway,2021/9/3,5421241,83%,30920.0,7273050.0,5.657,1330.689,30617.0,5.602,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.004980,0.004980
612,Norway,2021/9/4,5421241,83%,15887.0,7288937.0,2.907,1333.595,30519.0,5.584,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.004946,0.004946
613,Norway,2021/9/5,5421241,83%,15821.0,7304758.0,2.895,1336.490,30409.0,5.564,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.004914,0.004914
614,Norway,2021/9/6,5421241,83%,40079.0,7344837.0,7.333,1343.823,30243.0,5.533,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.004879,0.004879


In [6]:
df_Norway1.to_csv("df_Norway_after21days",index=False)